In [5]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from tweepy import Stream
from datetime import datetime, timedelta
from textblob import TextBlob

In [6]:
df = pd.read_csv("games.csv")
fd = pd.read_csv("games_details.csv")

C:\Users\dgold\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
mdf = pd.merge(fd, df[['GAME_ID', 'GAME_DATE_EST']], on='GAME_ID', how='left')
mdf.rename(columns={'GAME_DATE_EST': 'Corresponding_Date'}, inplace=True)
mdf.tail()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,Corresponding_Date
669555,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,19,...,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN,2012-10-06
669556,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,23,...,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN,2012-10-06
669557,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,15,...,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN,2012-10-06
669558,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,19,...,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN,2012-10-06
669559,11200005,1610612743,DEN,Denver,201951,Ty Lawson,NaN,NaN,NaN,27,...,2.0,2.0,6.0,2.0,0.0,6.0,1.0,8.0,NaN,2012-10-06


In [8]:
ACCESS_TOKEN = "925165003644293120-KrOGNXnlRbQMY1bFtJNCZWtuakW7bkq"
ACCESS_TOKEN_SECRET = "F1RLfhsZTyclZmXJts4MTKZCcbEjrtgHvTCN6ckNQJiBO"
CONSUMER_KEY = "cxX0fSaEXVvSrr7MVEYIIxWKM"
CONSUMER_SECRET = "86oxhhgE4rP0hnKFJMM8dWToPyJAG76tO9iwC0WFp2VxTFJR6S"

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

twitter_df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count',
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])


In [9]:
def stream(name):
    i = 0
    for tweet in tweepy.Cursor(api.search_tweets, q=name, count=10000,lang='en').items():
        print(i, end='\r')
        twitter_df.loc[i, 'Tweets'] = tweet.text
        twitter_df.loc[i, 'User'] = tweet.user.name
        twitter_df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        twitter_df.loc[i, 'user_followers'] = tweet.author.followers_count
        twitter_df.loc[i, 'User_location'] = tweet.user.location
        twitter_df.loc[i, 'User_verified'] = tweet.user.verified
        twitter_df.loc[i, 'fav_count'] = tweet.favorite_count
        twitter_df.loc[i, 'rt_count'] = tweet.retweet_count
        twitter_df.loc[i, 'tweet_date'] = tweet.created_at
          
        i+=1
        if i == 10000:
            break
        else:
            pass

In [10]:
def aggregateScore(name):
    score = 0
    i = 0
    for tweet in tweepy.Cursor(api.search_tweets, q=name, count=50,lang='en').items():
        blob = TextBlob(tweet.text)
        score += blob.sentiment.polarity
        i+=1
        if i == 50:
            break
        else:
            pass
    return score

In [6]:
twitter_df.drop(index=twitter_df.index, inplace=True)
#stream(data = ['Romeo Langford'])
stream("Kawhi Leonard")
twitter_df.to_csv("TweetExport.csv")


In [11]:
fin_df = pd.read_csv("Fin_Attributes.csv")
fin_df['Attribute'] = fin_df['PLAYER_NAME'].apply(aggregateScore)
mean = fin_df['Attribute'].mean()
std = fin_df['Attribute'].std()
fin_df['isOutlier'] = ['Yes' if x > mean + 2*std or x < mean - 2*std else 'No' for x in fin_df['Attribute']]
fin_df.to_csv("Filled_Attributes.csv")